In [1]:
import numpy as np
import tensorflow as tf

import logging
import datetime
import f90nml
import json

import matplotlib.pyplot as plt

from pinn_training_module import *

In [2]:
# Set backend for figures
%matplotlib qt

# Print tensorflow version
print('TensorFLow version: ', tf.__version__)

# Check if GPU is used
print(tf.config.list_physical_devices('GPU'))

# Enforce eager execution
# tf.config.run_functions_eagerly(True)

# Use float64 precision
tf.keras.backend.set_floatx('float64')

# Supress tensorflow warnings
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Template for printing training info to console
template = 'Epoch {:>4} | Train Loss: {:1.3e} | Test Loss: {:1.3e} | P Loss: {:1.3e} | Time (s): {}'

TensorFLow version:  2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Set up tensorboard

# Directory for logs
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
logdir = 'logs/' + timestamp

# Summary writer
summary_writer = tf.summary.create_file_writer(logdir)

In [4]:
with open('pinn_config.json', 'r') as f:
   pinn_config = json.load(f)['pinn_config']

pinn_config

{'N_multipoles': 7,
 'N_layers': 4,
 'N_neurons': 80,
 'N_points': 10000,
 'N_epochs': 1000,
 'N_renew': 500,
 'N_eval': 100,
 'N_Tcoeffs': 2,
 'optimizer': 'tf.keras.optimizers.Adam()',
 'loss_function': 'tf.keras.losses.MeanSquaredError()',
 'vacuum_mode': False}

In [5]:
# Create a test for validation

# Set multipole coeficients
c_l = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

# Set grid size
Nr = 100
Ntheta = 100

# Generate grid and test input
r, theta, X_test = generate_test(Nr, Ntheta, c_l)
# X_test, r, theta = jf.generate_test(Nr, Ntheta, c_l, N_multipoles)
q = 1 / r
mu = np.cos(theta)

# Calculate exact solution
P_exact = (1 - mu ** 2) * q
for l in range(2, len(c_l)+2):
   P_exact += (1 - mu ** 2) * (c_l[l-2] / l * P_l_prime (l, mu) * q ** (l))

P_exact = tf.reshape(P_exact, pinn_config['N_points'])

# RHS of the Grad-Shafranov equation
gs_exact = tf.zeros_like(P_exact)

In [6]:
# Initialize

# Create PINN
N = create_model(pinn_config, print_summary=False)

# # Generate initial input
# X = generate_input (N_points, N_multipoles)

# Compile train_step function to a tensoflow graph
train_step = tf.function(train_step, jit_compile=True)


# Initialise metrics
test_loss = tf.keras.metrics.MeanSquaredError()
P_abs_loss = tf.keras.metrics.MeanAbsoluteError()
P_rel_loss = tf.keras.metrics.MeanRelativeError(P_exact)
P_square_loss = tf.keras.metrics.MeanSquaredError()

P_metrics = [P_abs_loss, P_rel_loss, P_square_loss]
gs_metrics = [test_loss]


In [7]:
train_model (pinn_config, N
             , X_test, gs_metrics, P_metrics, gs_exact, P_exact
             , summary_writer, template
             , train_step
            )   

N.save('pinn_pde_solver.h5', include_optimizer=True)


Epoch    0 | Train Loss: 1.262e+01 | Test Loss: 1.016e+00 | P Loss: 4.271e-02 | Time (s): 11.043137788772583
Epoch  100 | Train Loss: 6.536e+00 | Test Loss: 9.542e-01 | P Loss: 3.530e-02 | Time (s): 5.5920023918151855
Epoch  200 | Train Loss: 2.227e+00 | Test Loss: 7.733e-01 | P Loss: 5.003e-02 | Time (s): 5.564822196960449


KeyboardInterrupt: 

In [ ]:
bla = tf.reshape(tf.range(20), (5,4))
bla[:,2:]

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[ 2,  3],
       [ 6,  7],
       [10, 11],
       [14, 15],
       [18, 19]], dtype=int32)>